# Business Context

## Implementing a RAG System for Power BI Usage

**Problem Scenario:**

In the current data-driven landscape, organizations increasingly rely on powerful analytics tools like Power BI to derive insights and make informed decisions. However, many analysts struggle with the complexity and breadth of Power BI’s official documentation. The extensive resources often lead to confusion, causing users to misinterpret features or overlook essential functionalities. This challenge can result in inefficient data analysis, wasted time, and missed opportunities for actionable insights. Consequently, analysts may not fully leverage the capabilities of Power BI, stifling potential business growth and impact.

# Objective

To address these challenges, we propose implementing a **Retrieval-Augmented Generation (RAG) system** specifically designed for Power BI. This system will enable analysts to formulate questions using natural language and retrieve concise, relevant answers directly from the official documentation. By facilitating better access to critical information, we aim to enhance the operational efficiency of analysts and empower them to utilize Power BI to its fullest potential.

The RAG application will simplify interactions with Power BI documentation, allowing users to inquire about specific features, functions, or best practices and receive clear explanations in real-time. By improving understanding and accessibility to the tool, analysts will be able to make quicker, data-driven decisions that lead to a significant business impact.

# Installing and Importing the Necessary Libraries

In this section, we need to install and import libraries required to run the notebook:

- The `openai` package provides the official OpenAI API client for accessing models like GPT-4, Whisper, DALL·E, including its embedding models

- The `tiktoken`	library provides access to OpenAI's tokenizer models, crucial for chunking and token counting

- The `pypdf` library parses and extracts text from PDF files — useful for document ingestion

- LangChain is a GenAI framework to build applications with LLMs using chains and agents.
  - `langchain` is the core library that provides access to various LangChain abstractions
  - `langchain-community` provides access to 3rd-party integrations (e.g., different vector stores, tools)
  - `langchain-chroma` provides specific integration to use ChromaDB as the vector store backend in LangChain
  - `langchain-openai` module provides a plug-in interface for LangChain to call OpenAI's LLMs using standardized interface

- `chromadb` library provides access to ChromaDB vector database, which is a fast, vector database optimized for retrieval in RAG systems

In [ ]:
# Installing the required libraries
!pip install -q openai==1.66.3 \
                tiktoken==0.9.0 \
                pypdf==5.4.0 \
                langchain==0.3.20 \
                langchain-community==0.3.19 \
                langchain-chroma==0.2.2 \
                langchain-openai==0.3.9 \
                chromadb==0.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.4/567.4 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.1/602.1 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.9/458.

**Importing the Libraries**


In [ ]:
# Importing the standard Libraries
import time                           # For measuring execution time or adding delays
from datetime import datetime         # For handling timestamps and datetime operations

# ChromaDB Vector Database
import chromadb  # Chroma: a local-first vector database for storing and querying document embeddings

# OpenAI SDK
from openai import OpenAI             # Official OpenAI Python SDK (v1.x) for interacting with models like GPT-4

# LangChain Utilities
# RecursiveCharacterTextSplitter intelligently breaks long text into smaller chunks with some overlap, preserving context.
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Loads all PDF files from a directory and extracts text from each.
from langchain_community.document_loaders import PyPDFDirectoryLoader

# Base class representing a document in LangChain; useful for downstream chaining and processing.
from langchain_core.documents import Document

# Embeddings and Vector Store
# Generates vector embeddings using OpenAI’s embedding models (e.g., `text-embedding-3-small`)
from langchain_openai import OpenAIEmbeddings

# Integration for using Chroma as the vector store within LangChain’s ecosystem
from langchain_chroma import Chroma

#ignore all warings
import warnings
warnings.filterwarnings('ignore')

#hide warnings from chroma
import logging
logging.getLogger("chromadb").setLevel(logging.CRITICAL)

## Setup the API Key
#### Setup the OpenAI API key and initialize the client with the required model.

In [ ]:
# Set up the OpenAI API Key
import os
from google.colab import userdata

openai_api_key = userdata.get('OPENAI_API_KEY')
base_url = 'https://aibe.mygreatlearning.com/openai/v1'

client = OpenAI(
    api_key = openai_api_key,
    base_url = base_url
)

model_name ='gpt-4o-mini'

## Creating Vector Database

In [ ]:
# Unzip the dataset containing the policy document
!unzip PowerBI.zip

Archive:  PowerBI.zip
  inflating: Introducing_Power_BI.pdf  


## Load PDF Documents and perform chunking

In this step, we need to:

- Load PDF documents from folder where pdf are saved using PyPDFDirectoryLoader.

- Split documents into chunks using RecursiveCharacterTextSplitter with the specified tokenizer, chunk size, and overlap.

- Store chunks within LangChain’s Document class.

- Inspect contents of the first page by accessing its .page_content attribute.

- Define a ChromaDB collection name to store the chunks for later retrieval.

In [ ]:
# Set the directory where PDF files to be stored
pdf_file_loc = '/content/Introducing_Power_BI.pdf'

#load pdf file
loader = PyPDFDirectoryLoader('/content/')

In [ ]:
#split document into chunks

def doc_spliter_(document, chunk_size, chunk_overlap):

  spliter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        encoding_name = 'cl100k_base',
        chunk_size = chunk_size,
        chunk_overlap = chunk_overlap
    )

  #load and split documents
  chunks = document.load_and_split(spliter)

  if chunks:

    for i, chunk in enumerate(chunks):
      print(f'\n --- chunk {i+1} --')
      print(f'producer:{chunk.metadata['producer']}')
      print(f'creator:{chunk.metadata['creator']}')
      print(f'creationdate:{chunk.metadata['creationdate']}')
      print(f'author:{chunk.metadata['author']}')
      print(f'title:{chunk.metadata['title']}')
      print(f'Source:{chunk.metadata['source']}')
      print(f'total_pages:{chunk.metadata['total_pages']}')
      print(f'page:{chunk.metadata['page']}')
      print(f'page_label:{chunk.metadata['page_label']}')
      print(f'Length: {len(chunk.page_content)} characters')
      print(f'content:')
      print(chunk.page_content)
      print('-'*40)

  return chunks


In [ ]:
# Chunks are stored within LangChain's Document class
chunked_docs = doc_spliter_(loader, 512, 16)


 --- chunk 1 --
producer:Adobe Acrobat Pro 10.1.16
creator:Adobe Acrobat Pro 10.1.16
creationdate:2016-06-13T10:18:21-04:00
author:Joan
title:
Source:/content/Introducing_Power_BI.pdf
total_pages:407
page:0
page_label:1
Length: 63 characters
content:
Introducing
Microsoft 
Power BI
Alberto Ferrari and Marco Russo
----------------------------------------

 --- chunk 2 --
producer:Adobe Acrobat Pro 10.1.16
creator:Adobe Acrobat Pro 10.1.16
creationdate:2016-06-13T10:18:21-04:00
author:Joan
title:
Source:/content/Introducing_Power_BI.pdf
total_pages:407
page:1
page_label:2
Length: 934 characters
content:
PUBLISHED BY 
Microsoft Press 
A division of Microsoft Corporation 
One Microsoft Way 
Redmond, Washington 98052-6399 
Copyright © 2016 by Microsoft Corporation 
All rights reserved. No part of the contents of 
this book may be reproduced or transmitted in 
any form or by any means without the written 
permission of the publisher. 
ISBN: 978-1-5093-0228-4 
Microsoft Press books are avail

In [ ]:
# Inspecting the contents of the first page by accessing its .page_content attribute
chunked_docs[0].page_content

'Introducing\nMicrosoft \nPower BI\nAlberto Ferrari and Marco Russo'

In [ ]:
# define the ChromaDB collection name to store the chunks
powerBI_collection = 'powerbi_manual'

### Initialize the OpenAI embedding model with the API key, endpoint, and embedding model name.
In this step, we need to:

- Instantiate the OpenAI embedding model with the API key, endpoint, and embedding model name.

- Initialize a persistent Chroma client for managing embeddings.

- Ping the database client using the heartbeat method to confirm the connection is alive.

- Verify the database is empty before adding new embeddings.

- Create a Chroma vector store to store and retrieve document embeddings.

- Confirm the collection creation and that the database has been populated.

- Batch process 500 chunks at a time when sending to the API, and pause execution for 30 seconds after each batch to avoid rate limits.


In [ ]:
# Instantiate the OpenAI embedding model
embedding_model = OpenAIEmbeddings(
    api_key = openai_api_key,
    base_url = base_url,
    model = 'text-embedding-3-small'
)

In [ ]:
# Initialize a persistent Chroma client

local_path = './powerBI_db' #define the local directory where the database will be stored
chromadb_client = chromadb.PersistentClient(
    path = local_path
)

In [ ]:
# Pinging the database client to check if the connection is alive
# the heartbeat method returns the current time in nanoseconds and is generally used to check if the server is alive
chromadb_client.heartbeat()

1768686237068789825

In [ ]:
# Confirm database is empty
chromadb_client.count_collections()

0

In [ ]:
# Instantiate a Chroma vector store to store and retrieve document embeddings
def vector_store(chunks, persist_directory = local_path):

  #create a chromaBD vector store
  vector_store = Chroma(
      collection_name = powerBI_collection,
      collection_metadata = {'hnsw:space': 'cosine'},
      embedding_function = embedding_model,
      client = chromadb_client,
      persist_directory = local_path
  )

  return vector_store

print(f'Vector store created and saved to {local_path}')
vectorstore = vector_store(chunked_docs)

Vector store created and saved to ./powerBI_db


In [ ]:
# confirm collection creation
chromadb_client.list_collections()

['powerbi_manual']

In [ ]:
# Confirm database has been populated with the collection
chromadb_client.count_collections()

1

In [ ]:
# Batch 500 chunks to send to the API at a time, pausing execution for 30 seconds afterward
for i in range(0, len(chunked_docs), 500):
  vectorstore.add_documents(
      documents = chunked_docs[i:i+500],
      ids =['text_'+ str(i) for i in range(i, i+500)]
  )

  time.sleep(30)



# CRUD Operations in ChromaDB




## **READ**

Once the database is created, the stored entries can be retrieved by initializing a new Chroma instance (denoted as **vectorstore_persisted** to distinguish between creation and read operations) and directing it to the persistent storage directory containing the document embeddings.

In this step, we need to:

Initialize a new Chroma instance (e.g., vectorstore_persisted) and point it to the persistent storage directory where embeddings are stored.

In [ ]:
vectorstore_persisted =Chroma(
    collection_name = powerBI_collection,
    collection_metadata={'hnsw:space':'cosine'},
    embedding_function= embedding_model,
    client = chromadb_client,
    persist_directory = local_path
)

There are two valuable types of READ operations in vector databases:

1. **Inspecting individual records**
2. **Retrieving relevant records based on a user query**

In this step, we need to:

Define the Chroma collection to work with.

Count the number of records present in the collection.

Inspect the first 2 records using the .peek() method to confirm that embeddings have been stored correctly.

**Inspecting individual records**

In [ ]:
# Define the chroma collection
collection = chromadb_client.get_collection(powerBI_collection)

In [ ]:
# Count the number of records in the collection
collection.count()

407

In [ ]:
# Inspect the first 2 records using the .peek() method
collection.peek(2)

{'ids': ['text_0', 'text_1'],
 'embeddings': array([[-0.0141233 , -0.03104495,  0.0387464 , ..., -0.00077508,
          0.00032513,  0.01716083],
        [ 0.0097238 ,  0.00484162,  0.02150327, ..., -0.019948  ,
         -0.01871731,  0.01614773]]),
 'documents': ['Introducing\nMicrosoft \nPower BI\nAlberto Ferrari and Marco Russo',
  'PUBLISHED BY \nMicrosoft Press \nA division of Microsoft Corporation \nOne Microsoft Way \nRedmond, Washington 98052-6399 \nCopyright © 2016 by Microsoft Corporation \nAll rights reserved. No part of the contents of \nthis book may be reproduced or transmitted in \nany form or by any means without the written \npermission of the publisher. \nISBN: 978-1-5093-0228-4 \nMicrosoft Press books are available through \nbooksellers and distributors worldwide. If you \nneed support related to this book, email \nMicrosoft Press Support at \nmspinput@microsoft.com. Please tell us what \nyou think of this book at http://aka.ms/tellpress. \nThis book is provided “as-

**Retrieving relevant records based on a user query**

The primary function of the vector database is to retrieve relevant records based on user queries and to facilitate this process, we implement a retriever that utilizes the query embeddings to query the database.

Now we are going to write a code that uses HNSW algorithm to calculate the nearest neighbors for the user query and returns the corresponding documents from the database.

In [ ]:
# Create a retriever interface from the vector store
retriever = vectorstore_persisted.as_retriever(
    search_type = 'similarity',
    search_kwargs={'k':5}
    )

In [ ]:
# Define a sample user query
user_query = 'How to access the power query editor in Power BI?'

code to perform the similarity search based on the user query by using the `.invoke()` method.

In [ ]:
# Perform similarity search to return the top 5 document chunks based on the sample user query
# Replace 'user_query' with the actual query you want to search for

relevant_docs = retriever.invoke(user_query)

print(f'User query: {user_query} \n')

for i, doc in enumerate(relevant_docs, 1):
  print(f'Document {i} \n{doc.page_content} \n')


User query: How to access the power query editor in Power BI? 

Document 1 
143 C H A P T E R  4  |  Using Power BI Desktop 
 
Query Editor opens in a new window, presenting 
myriad options, as depicted in Figure 4-6. 
 
Figure 4-6: Power BI Desktop’s Query Editor is a 
complete development environment in and of itself. 
Let’s take a quick tour of the Query Editor 
window. Along the top is the ribbon, which has 
four tabs: Home, Transform, Add Column, and 
View. Below the ribbon, on the left side, is the 
Query pane, which displays a list of all the 
queries for the model. The middle pane shows 
the result of the query. The Query Settings pane 
on the right displays the query properties. 
In David’s scenario, he is already accessing the 
2015 sales data from the Contoso database, so 
the objective now is to create a new query that 

Document 2 
142 C H A P T E R  4  |  Using Power BI Desktop 
 
The query language of Power BI Desktop is used 
by Query Editor, and discussion of that lang

## Observations
The system is retrieving relevant chunks according to the user query. These chunks will be fed to the model to answer user queries accordengly.


# RAG Q&A System for PowerBI Documentation

A typical RAG implementation consists of the following stages:
* Indexing Stage
* Retrieval Stage
* Generation Stage

| Stage          | Key Activities                                        | Role in RAG                              |
| -------------- | ----------------------------------------------------- | ---------------------------------------- |
| **Indexing**   | Chunking · Embedding · Storing                        | Prepares data for efficient retrieval    |
| **Retrieval**  | Query embedding · Similarity search   | Consolidates relevant context            |
| **Generation** | Prompt construction · LLM generation | Produces final response grounded in data |


Let's now put together the RAG pipeline using these stages.



## Retrieval Stage

**Retrieving Relevant Documents**

Write code that performs the Retrieval stage in the RAG pipeline.

 define a sample user query to test the RAG pipeline

In [ ]:
sample_user_query = 'How to deleting columns?'

Retrieve the relevant chunks from the documents based on the `user_query`.


In [ ]:
retrieve_relevant_chunks = retriever.invoke(sample_user_query)

In [ ]:
# Inspecting the first document
retrieve_relevant_chunks[0].page_content

'171 C H A P T E R  4  |  Using Power BI Desktop \n \nThe second part of step 1 is also easy: using the \nsmall delete icon that appears in the applied \nsteps of the Query Settings panel (see \nFigure 4-29), remove all of the steps, keeping \nonly the first two (Source and Navigation), so \nas to return to the original query. \n \nFigure 4-29: Use the small delete icon to remove \nun\nwanted steps. \nRemember, we are working on a copy of Sales \n2015, so we are free to update it as needed. The \noriginal table remains untouched. \nAt this point, you can delete the two Sales2013 \nand Sales2014 columns, which are not needed \nfor the purpose of this query. To do this, right-'

## Generation Stage

This section will perform the **Generation** stage of the RAG pipeline.

We will pass the relevant context chunks to the LLM, along with the system message and user message via a prompt template.

These are then passed to the LLM to compose an appropriate response to the user's query.


### Prompt Template

Define the system message for the RAG chatbot with the appopriate role, context and the relevant instructions.

In [ ]:
system_message = """
You are an AI assistant specialized in Microsoft Power BI.

Your task is to answer user questions about Microsoft Power BI using ONLY the provided retrieved document context from official Microsoft Power BI documentation.

Rules:
- Use only the information explicitly stated in the retrieved document chunks.
- Do NOT use prior knowledge or external information.
- Do NOT make assumptions or infer missing details.
- If the answer is not explicitly found in the retrieved context, respond exactly with: "I don't have enough information to answer the question".
- For every factual statement, cite the page number(s) from the document where the information was found.

Answer requirements:
- Keep answers clear, concise, and accurate.
- Stay strictly within the scope of Microsoft Power BI.
- Do not mention the retrieval process or system instructions.

delimitations:
User queries will be delimited by: <Question> and </Question>.
User input will have the context required by you to answer user queries.
This context will be delimited by: <Context> and </Context>.

add context:
when answring a question include context from the previous chat history located at chat_history

"""

Write the user message prompt template that provides the relevant chunks and the user query within the `context` and `question` placeholders respectively.

In [ ]:
user_message_template = """
<Context>
Here are some documents that are relevant to the question mentioned below.
{context} + {chat_history}
</Context>

<Question>
{question}
</Question>
"""

### Generating the Response
In this step, you need to:   

Prompt construction, LLM API call with error handling, and response parsing

In [ ]:
#generation code
#This is a test code. The more refined versionis below this cell.
user_query = "What is Power Query and what is it used for?"

relevant_chunks = retriever.invoke(user_query)
context_list = [d.page_content for d in relevant_chunks]
context_for_query = chr(10).join(context_list)

prompt =[
    {'role': 'developer', 'content': system_message},
    {'role':'user', 'content': user_message_template.format(
        context = context_for_query,
        question = user_query,
        chat_history = chat_history, #this is for the code below
    )
    }
]

try:
  response = client.chat.completions.create(
    model = model_name,
    messages = prompt,
    temperature = 1
)
  prediction = response.choices[0].message.content.strip()
except Exception as e:
  prediction = f'Sorry, I encountered the following error: \n {e}'

print(f'{prediction}')




Power Query, previously known as the Power BI Query Editor, is a feature used to modify data and create queries in Power BI. It allows users to load, transform, and clean data from various sources before performing analysis. To access Power Query in Power BI Desktop, users can click on "Edit Queries" in the Home tab of the ribbon (page 142).


In [ ]:
#This is going to be our first version of a chatbot

#create an empty list to store the chat history
chat_history = []

#define a user query function
def main():
  while True:
    user_query = str(input("Ask me any question or enter 'quit' to quit:").strip())

    if user_query.lower() == 'quit':
      print('See you later!')
      break
    else:
      chat_bot(user_query)

      chat_history.append(prediction)

#create a function to generate the answer
def chat_bot(user_query):

  relevant_chunks = retriever.invoke(user_query)
  context_list = [d.page_content for d in relevant_chunks]
  context_for_query = chr(10).join(context_list)

  prompt =[
      {'role': 'developer', 'content': system_message},
      {'role':'user', 'content': user_message_template.format(
          context = context_for_query,
          question = user_query,
          chat_history = chat_history
      )
      }
  ]

  try:
    response = client.chat.completions.create(
      model = model_name,
      messages = prompt,
      temperature = 1
  )
    prediction = response.choices[0].message.content.strip()
  except Exception as e:
    prediction = f'Sorry, I encountered the following error: \n {e}'

  return print(prediction)

main()




Ask me any question or enter 'quit' to quit:what is Power BI?
Power BI is a cloud service that provides tools to perform analysis of data and gain insights from your numbers. It includes the capability to build dashboards, which require a dataset, a report, and the dashboard itself. Datasets are the source of data, reports allow for the creation of visualizations that might be interconnected, and a dashboard serves as a collection of visualizations and/or reports (page 55).
Ask me any question or enter 'quit' to quit:How to install it?
To install Power BI Desktop, you need to follow these steps: 

1. On the Power BI website, click the download button on the right side of the menu bar at the top (see step 1 in Figure 3-3).
2. Then, click on Power BI Desktop (see step 2).
3. After downloading Power BI Desktop, install it by following the instructions provided in the Microsoft Power BI Desktop Setup Wizard.
4. Once installed, start the application. A welcome screen will greet you, followe

## Observation
The code abode generates the answer and keeps the context of the previous response. This helps us ask subsequent questions using pronouns like it. The model already know what we are refering to, since we saved the context of the previous chat. This is exactly how chat GPT works.

# Putting it all together - PowerBI RAG Q&A Chatbot

We'll now we will put together the relevant codes for the RAG pipeline into a file named `rag-chat.py` to create a basic command-line chat interface which can run via  the terminal. This will be the second version of our chatbot

This naive RAG implementation illustrates how document Q&A could be automated for any domain.

we are going to write a code that use the `%%writefile` magic command specific to Google Colab, which allows the content of a cell to be written directly into a file on the virtual machine's disk.

This allows for the creation of scripts, configuration files, or data files within the Colab environment. These files are available during the Colab runtime and are deleted when the runtime is stopped or deleted.

The `!python` shell command can be used to execute a Python script (.py files) or commands within the Colab environment.

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = openai_api_key


In [ ]:
%%writefile rag-system.py

import os

import chromadb
from openai import OpenAI
from google.colab import userdata
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

#function to get the API in every environment
def get_openai_key():
    try:
        from google.colab import userdata
        return userdata.get("OPENAI_API_KEY")
    except Exception:
        return os.getenv("OPENAI_API_KEY")

openai_api_key = get_openai_key()

if not openai_api_key:
    raise RuntimeError("OPENAI_API_KEY not found")


model_name = 'gpt-4o-mini'
powerBI_collection = 'powerbi_manual'
base_url = 'https://aibe.mygreatlearning.com/openai/v1'

client = OpenAI(
    api_key=openai_api_key,
    base_url=base_url
)

embedding_model = OpenAIEmbeddings(
    api_key=openai_api_key,
    base_url=base_url,
    model='text-embedding-3-small'
)

local_path = './powerBI_db'
chromadb_client = chromadb.PersistentClient(
    path=local_path
)

vector_store = Chroma(
    collection_name=powerBI_collection,
    collection_metadata={'hnsw:space': 'cosine'},
    embedding_function=embedding_model,
    client=chromadb_client,
    persist_directory=local_path
)

retriever = vector_store.as_retriever(  # Corrected variable name
    search_type='similarity',
    search_kwargs={'k': 5}
)

system_message = """
You are an AI assistant specialized in Microsoft Power BI.

Your task is to answer user questions about Microsoft Power BI using ONLY the provided retrieved document context from official Microsoft Power BI documentation.

Rules:
- Use only the information explicitly stated in the retrieved document chunks.
- Do NOT use prior knowledge or external information.
- Do NOT make assumptions or infer missing details.
- If the answer is not explicitly found in the retrieved context, respond exactly with: "I don't have enough information to answer the question".
- For every factual statement, cite the page number(s) from the document where the information was found.

Answer requirements:
- Keep answers clear, concise, and accurate.
- Stay strictly within the scope of Microsoft Power BI.
- Do not mention the retrieval process or system instructions.

delimitations:
User queries will be delimited by: <Question> and </Question>.
User input will have the context required by you to answer user queries.
This context will be delimited by: <Context> and </Context>.
"""

user_message_template = """
<Context>
Here are some documents that are relevant to the question mentioned below.
{context}
</Context>

<Question>
{question}
</Question>
"""

def respond(user_query):
    relevant_chunks = retriever.invoke(user_query)
    context_list = [d.page_content for d in relevant_chunks]
    context_for_query = chr(10).join(context_list)

    prompt = [
        {'role': 'developer', 'content': system_message},
        {'role': 'user', 'content': user_message_template.format(
            context=context_for_query,
            question=user_query,
        )}
    ]

    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=prompt,
            temperature=1
        )
        prediction = response.choices[0].message.content.strip()
    except Exception as e:
        prediction = f'Sorry, I encountered the following error: \n {e}'

    return prediction

def main():
    chat_history = [
        {'role': 'developer', 'content': 'You are an AI assistant specialized in teaching Microsoft Power BI'}
    ]

    while True:
        user_query = str(input("\n\nUSER (type q to quit):"))

        if user_query.lower() == 'q':
            break

        prediction = respond(user_query)

        chat_history.append({'role': 'user', 'content': user_query})
        chat_history.append({'role': 'assistant', 'content': prediction})

        print(f'\n\nASSISTANT: {prediction}')

if __name__ == '__main__':
    main()


Overwriting rag-system.py


Run the script using the `!python` shell command.

We are going to fomulate 5 queries on the PowerBI Documentation that will then be used to validate the the Q&A RAG Chatbot and provide the output responses.

# User queries
* How can I import data from an Excel file into Power BI Desktop?
* How do I create relationships between tables?
* Explain the steps to create a measure using DAX.
* What does the CALCULATE function do in DAX?
* How do I change data types in Power BI?

In [ ]:
!python rag-system.py

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


USER (type q to quit):How can I import data from an Excel file into Power BI Desktop?
Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


ASSISTANT: To import data from an Excel file into Power BI Desktop, you need to click the Import button after selecting the Excel file stored in OneDrive for Business. This allows you to import the content of the Excel file. You have two options at this stage: you can either import the data as a raw data source for your reports or choose to connect, manage, and view the Excel file in Power BI. Selecting "Import Excel Data Into Power BI" will bring the data into your Power BI Desktop model (page 84).


USER (type q to quit):How do I create relationships between tables?


ASSIST